<a href="https://colab.research.google.com/github/Christi049/grocery-saver-agent/blob/main/grocery_saver_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

file = json.load(open("dataset.json", "r"))
print(file[1])

{'instruction': 'Plan a 7-day dinner menu minimizing food waste and reusing ingredients across dishes.', 'input': {'dietary_restriction': 'None', 'ingredients': [{'name': 'Chickpeas', 'expiry': '2025-09-03'}, {'name': 'Bell peppers', 'expiry': '2025-09-04'}, {'name': 'Yogurt', 'expiry': '2025-09-06'}, {'name': 'Rice', 'expiry': '2025-09-20'}]}, 'output': {'meals': {'Monday': 'Chickpea curry with rice', 'Tuesday': 'Stuffed bell peppers with chickpeas and yogurt dip', 'Wednesday': 'Rice bowl with roasted bell peppers, chickpeas, and yogurt sauce', 'Thursday': 'Chickpea and yogurt salad with roasted peppers', 'Friday': 'Spiced rice pilaf with chickpeas and peppers', 'Saturday': 'Chickpea patties with yogurt dipping sauce and rice', 'Sunday': 'Bell pepper stir-fry with chickpeas and yogurt drizzle'}, 'shopping_list': ['Garlic', 'Cumin', 'Cilantro']}}


In [2]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [3]:
# Only if you want to include your existing examples
with open("dataset.json", "r") as f:
    current_examples = json.load(f)

In [4]:
ingredients = ["eggs", "cheese", "sausage", "spinach", "mushrooms"]
methods = ["scrambled", "grilled", "baked", "sautéed"]
meals = ["omelette", "wrap", "muffin", "skillet"]
dietary_options = ["None", "Vegetarian", "Vegan", "Gluten-Free"]

new_examples = []

for method in methods:
    for meal in meals:
        for i in ingredients:
            for j in ingredients:
                if i != j:
                    dietary = dietary_options[len(new_examples) % len(dietary_options)]
                    instruction = "Suggest creative breakfast options using items that will expire soon."
                    input_data = {
                        "dietary_restriction": dietary,
                        "ingredients": [
                            {"name": i, "expiry": "2025-09-03"},
                            {"name": j, "expiry": "2025-09-03"}
                        ]
                    }
                    output_data = {
                        "meals": {
                            "Monday": f"{method} {i} and {j} {meal}",
                            "Tuesday": f"{method} {j} and {i} {meal}",
                            "Wednesday": f"{method} {i} {meal} with {j}",
                            "Thursday": f"{method} {j} {meal} with {i}",
                            "Friday": f"{method} {i} and {j} {meal} wrap",
                            "Saturday": f"{method} {i} and {j} {meal} skillet",
                            "Sunday": f"{method} {i} and {j} {meal} muffin"
                        },
                        "shopping_list": ["Tomatoes", "Onions", "Bell peppers"]
                    }
                    new_examples.append({
                        "instruction": instruction,
                        "input": input_data,
                        "output": output_data
                    })

print(f"Generated {len(new_examples)} new examples")

Generated 320 new examples


In [5]:
# Only if you loaded current_examples
combined_dataset = current_examples + new_examples

In [14]:
# Save new dataset only
with open("dataset.json", "w") as f:
    json.dump(new_examples, f, indent=2)

# Save combined dataset (if combined)
with open("combined_dataset.json", "w") as f:
    json.dump(combined_dataset, f, indent=2)


In [7]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [15]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [17]:
import json
from datasets import Dataset

def format_prompt(example):
    return (
        f"### Instruction: {example['instruction']}\n"
        f"### Input: {json.dumps(example['input'], ensure_ascii=False)}\n"
        f"### Output: {json.dumps(example['output'], ensure_ascii=False)}<|endoftext|>"
    )

# Use your combined_dataset instead of 'file'
formatted_data = [format_prompt(item) for item in combined_dataset]

# Create a Hugging Face Dataset
dataset = Dataset.from_dict({"text": formatted_data})

In [18]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

In [19]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/369 [00:00<?, ? examples/s]

In [20]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 369 | Num Epochs = 3 | Total steps = 141
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 97,255,424 of 3,310,005,248 (2.94% trained)


Step,Training Loss,entropy
25,0.488000,0
50,0.090300,No Log
75,0.085500,No Log
100,0.074900,No Log
125,0.050500,No Log


In [22]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt (dataset format)
test_instruction = "Generate a vegan 7-day meal plan that prioritizes ingredients nearing expiry."
test_input = {
    "dietary_restriction": "Vegan",
    "ingredients": [
        {"name": "Zucchini", "expiry": "2025-08-08"},
        {"name": "Rice", "expiry": "2025-08-10"},
        {"name": "Almonds", "expiry": "2025-08-22"}
    ]
}

messages = [
    {"role": "user", "content": f"### Instruction: {test_instruction}\n### Input: {json.dumps(test_input)}\n### Output:"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
response = response.split("<|endoftext|>")[0]
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 01 Sep 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

### Instruction: Generate a vegan 7-day meal plan that prioritizes ingredients nearing expiry.
### Input: {"dietary_restriction": "Vegan", "ingredients": [{"name": "Zucchini", "expiry": "2025-08-08"}, {"name": "Rice", "expiry": "2025-08-10"}, {"name": "Almonds", "expiry": "2025-08-22"}]}
### Output:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

### Input: {"dietary_restriction": "Vegan", "ingredients": [{"name": "Zucchini", "expiry": "2025-08-08"}, {"name": "Rice", "expiry": "2025-08-10"}, {"name": "Almonds", "expiry": "2025-08-22"}]}
### Output: {"meals": {"Monday": "Zucchini stir-fry with rice and almonds", "Tuesday": "Rice bowl with sautéed zucchini and almonds", "Wednesday": "Zucchini and rice pilaf with almonds", "Thursday": "Vegan zucchini burgers on rice with almonds", "Friday": "Rice and almond 

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)